<a href="https://colab.research.google.com/github/hamnasghar/Embedding_Assignment6/blob/main/Assignment_6_Hamna_Asghar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install -U sentence-transformers

In [28]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import xgboost as xgb

In [4]:
# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

# For retrieval you need to pass this prompt.
query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [23]:
df = pd.read_csv('Embeddings.csv')
print(df)

        Unnamed: 0                                               text  \
0                0                                               text   
1                1  rising cases of covid does not alarm me rising...   
2                2  please vote for chicagoindiaresolution marking...   
3                3  wishing all of you eidaladha hazrat ibrahim as...   
4                4  daily coronavirus cases in india top for first...   
...            ...                                                ...   
134343      134343  happydiwali to india from dgisi and dgispr you...   
134344      134344  india has been involved in many countries with...   
134345      134345  does anybody know website which ships pre orde...   
134346      134346  what hinted few weeks back in an interview wit...   
134347      134347  this is just slice of what we have pakistan na...   

        sentiment  
0               0  
1               1  
2               0  
3               1  
4               1  
...

In [25]:
# Missing Values drop
df = df.dropna(subset=['text'])
# Convert into strings
df['text'] = df['text'].astype(str)

<ipython-input-25-8aa4068fe409>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


In [26]:
# 2. Encode
embeddings = model.encode(df['text'].tolist())

# Optional: Quantize the embeddings
binary_embeddings = quantize_embeddings(embeddings, precision="ubinary")

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)



similarities: tensor([[0.4062, 0.4376, 0.4300,  ..., 0.4130, 0.3620, 0.3715]])


In [29]:
# Prepare the data
labels = df['sentiment'].values
X = np.array(binary_embeddings)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Create an XGBoost classifier
classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Predict the sentiments for the testing set
y_pred = classifier.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7308866224968361
Precision: 0.7442296163069544
Recall: 0.7223596159441373
F1 Score: 0.7331315517495941
